# TP 3: Chatbot à Règles, Intention + Mémoire de Contexte

## Objectifs du TP
- Implémenter un agent symbolique (sans apprentissage automatique)
- Détecter des intentions à partir de mots-clés
- Gérer une mémoire de contexte (dernier sujet abordé)
- Concevoir des modules distincts : détection, mémoire, génération de réponses
- Comprendre les limitations des systèmes à règles

In [1]:
import regex, random
from datetime import datetime

## Partie I: Intention et Mots-Clés

Dans cette première partie, vous allez définir les intentions que votre chatbot peut reconnaître et les mots-clés associés à chaque intention.

**Q 1**: Définition du dictionnaire INTENTS

In [2]:
INTENTS = {
    "salutation": ["bonjour","salut","coucou","bonsoir","hey"],
    "meteo": ["meteo","temps","chaud","froid","pluie","soleil","automne","ete","printemps","hiver"],
    "heure": ["matin","soir","après-midi","heure","minute","seconde","temps", "aujourdhui", "demain", "hier"],
    "identite": ["qui","nom","objectif","but","pourquoi"],
    "aurevoir": ["bye","au revoir","ciao","à plus","à bientot","à demain"],
}

## Partie II: Normalisation du Texte
La normalisation du texte est une étape cruciale pour améliorer la robustesse de la détection  d'intentions. Elle permet de traiter uniformément les variations d'écriture. 


**Q 2**: Implémentation de la fonction normalize

In [3]:
accent_dict = {
    "a": ["à","â"],
    "e": ["é","è","ê","ë"],
    "u": ["ù","û","ü"],
    "o": ["ô"],
    "i": ["ï"],
    "c": ["ç"]
}

accent_map = {ch: base for base, lst in accent_dict.items() for ch in lst}

def normalize(text):

    text = str(text).lower()
    if accent_map:
        pattern = "[" + "".join(regex.escape(c) for c in accent_map.keys()) + "]"
        text = regex.sub(pattern, lambda m: accent_map[m.group(0)], text)

    # remove punctuation/symbols, collapse whitespace, strip
    text = regex.sub(r"[\p{P}\p{S}]+", " ", text)
    text = regex.sub(r"\s+", " ", text).strip()

    return text

In [4]:
# Contenu de accent_map
accent_map

{'à': 'a',
 'â': 'a',
 'é': 'e',
 'è': 'e',
 'ê': 'e',
 'ë': 'e',
 'ù': 'u',
 'û': 'u',
 'ü': 'u',
 'ô': 'o',
 'ï': 'i',
 'ç': 'c'}

## Partie 3: Détection d'Intention
Maintenant que vous avez défini les intentions et la normalisation, vous allez implémenter la  fonction qui détecte l'intention d'un message utilisateur. 


**Q 3**: Implémentation de detect_intent

In [5]:
def detect_intent(message):
    new_intent_table = {word:intent for intent, words in INTENTS.items() for word in words}
    message = normalize(message)
    for word in message.split(sep=" "):
        if word in new_intent_table:
            return new_intent_table[word]
        else:
            continue
    return "inconnu"
    

In [6]:
# new_intent_table
new_intent_table = {word:intent for intent, words in INTENTS.items() for word in words }
(new_intent_table)

{'bonjour': 'salutation',
 'salut': 'salutation',
 'coucou': 'salutation',
 'bonsoir': 'salutation',
 'hey': 'salutation',
 'meteo': 'meteo',
 'temps': 'heure',
 'chaud': 'meteo',
 'froid': 'meteo',
 'pluie': 'meteo',
 'soleil': 'meteo',
 'automne': 'meteo',
 'ete': 'meteo',
 'printemps': 'meteo',
 'hiver': 'meteo',
 'matin': 'heure',
 'soir': 'heure',
 'après-midi': 'heure',
 'heure': 'heure',
 'minute': 'heure',
 'seconde': 'heure',
 'aujourdhui': 'heure',
 'demain': 'heure',
 'hier': 'heure',
 'qui': 'identite',
 'nom': 'identite',
 'objectif': 'identite',
 'but': 'identite',
 'pourquoi': 'identite',
 'bye': 'aurevoir',
 'au revoir': 'aurevoir',
 'ciao': 'aurevoir',
 'à plus': 'aurevoir',
 'à bientot': 'aurevoir',
 'à demain': 'aurevoir'}

## Partie IV: Génération de Réponses + Mémoire
C'est la partie la plus importante du TP ! Vous allez implémenter la fonction qui génère les réponses en fonction de l'intention détectée et qui gère la mémoire de contexte. 

**Q 4**: Implémentation de respond avec mémoire

In [7]:
time_ids = INTENTS["heure"]
time_dict = {time:time for time in time_ids}
villes = {
    "casablanca": "casablanca",
    "rabat": "rabat",
    "fes": "fes",
    "marrakech": "marrakech",
    "agadir": "agadir",
    "tanger": "tanger"
}

FAKE_WEATHER = {
    "casablanca": 23,
    "rabat": 20,
    "fes": 24,
    "marrakech": 28,
    "agadir": 22,
    "tanger": 19
    }

In [8]:
def respond(intent, message, memory):
    message = normalize(message)

    match intent:
        case "salutation":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return random.choice(INTENTS[intent])
        
        case "aurevoir":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return random.choice(INTENTS[intent])
        
        case "heure":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            now = datetime.now()
            return f"Il est {now.hour}:{now.minute:02d}"
        
        case "identite":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return "Bonjour, je suis votre Agent IA, capable de converser avec vous sans aucun soucis."
        
        case "meteo":
            ville = [villes[city] for city in message.split(sep=" ") if city in villes]
            if not ville:
                ville = "casablanca"
            else:
                ville = ville[0]
            memory["last_intent"] = intent
            memory["last_city"] = ville

            return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
        
        case "inconnu":
            if memory.get("last_intent") == "meteo":
                time_ids = INTENTS["heure"] 
            
                ville = [villes[city] for city in message.split(sep=" ") if city in villes]
                heure = [time_dict[heure] for heure in message.split(sep=" ") if heure in time_ids]

                if not ville:
                    ville = memory.get("last_city", "casablanca")
                else:
                    ville = ville[0]
                
                if heure:
                    return f"{ville} sera à {FAKE_WEATHER[ville]} {heure[0]}."
                else:
                    return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
            
            return "Désolé, je n'ai pas compris votre demande."

## Partie V: Boucle de Dialogue Interactive 
Maintenant que toutes les fonctions sont implémentées, vous allez créer une boucle  interactive permettant de discuter avec le chatbot. 


**Q 5**: Implémentation de chat_loop

In [9]:
def chat_loop():
    memory = {}
    print("Chatbot: Bonjour! Je suis votre assistant. Tapez 'exit', 'quit' ou 'bye' pour quitter.")

    while True:
        user_input = input("Vous: ").strip()
        
        if not user_input:
            continue
        
        normalized_input = normalize(user_input)
        if normalized_input in ["exit", "quit", "bye"]:
            print("Chatbot: Au revoir!")
            break
        
        intent = detect_intent(user_input)
        response = respond(intent, user_input, memory)
        
        print(f"Chatbot: {response}")

In [10]:

memory = {}

message_1 = "Bonjour"
intent1 = detect_intent(message_1)
response1 = respond(intent1, message_1, memory)
print(f"Vous: {message_1}")
print(f"Chatbot: {response1}")
print(f"Memory: {memory}\n")

message_2 = "Quelle heure est-il ?"
intent2 = detect_intent(message_2)
response2 = respond(intent2, message_2, memory)
print(f"Vous: {message_2}")
print(f"Chatbot: {response2}")
print(f"Memory: {memory}\n")

message_3 = "La météo à Casablanca"
intent3 = detect_intent(message_3)
response3 = respond(intent3, message_3, memory)
print(f"Vous: {message_3}")
print(f"Chatbot: {response3}")
print(f"Memory: {memory}\n")

message_4 = "Et demain ?"
intent4 = detect_intent(message_4)
response4 = respond(intent4, message_4, memory)
print(f"Vous: {message_4}")
print(f"Chatbot: {response4}")
print(f"Memory: {memory}\n")

message_5 = "Qui es-tu ?"
intent5 = detect_intent(message_5)
response5 = respond(intent5, message_5, memory)
print(f"Vous: {message_5}")
print(f"Chatbot: {response5}")
print(f"Memory: {memory}\n")

message_6 = "Au revoir"
intent6 = detect_intent(message_6)
response6 = respond(intent6, message_6, memory)
print(f"Vous: {message_6}")
print(f"Chatbot: {response6}")
print(f"Memory: {memory}")

Vous: Bonjour
Chatbot: coucou
Memory: {'last_intent': 'salutation', 'last_city': 'casablanca'}

Vous: Quelle heure est-il ?
Chatbot: Il est 22:16
Memory: {'last_intent': 'heure', 'last_city': 'casablanca'}

Vous: La météo à Casablanca
Chatbot: casablanca est à 23 actuellement.
Memory: {'last_intent': 'meteo', 'last_city': 'casablanca'}

Vous: Et demain ?
Chatbot: Il est 22:16
Memory: {'last_intent': 'heure', 'last_city': 'casablanca'}

Vous: Qui es-tu ?
Chatbot: Bonjour, je suis votre Agent IA, capable de converser avec vous sans aucun soucis.
Memory: {'last_intent': 'identite', 'last_city': 'casablanca'}

Vous: Au revoir
Chatbot: Désolé, je n'ai pas compris votre demande.
Memory: {'last_intent': 'identite', 'last_city': 'casablanca'}


## Partie VI: Questions de Réflexion

Cette dernière partie vous invite à réfléchir sur les concepts abordés et les limitations de  l'approche par règles. 


**Q 6.1**: Différence intention vs réponse

Une intention permet de cerner le sujet concerné dans la réponse du chatbot, la réponse en elle meme est just un message qui vient résultant de l'intention précisée. On sépare ces deux concepts pour faire la différence entre ce que le chatbot dit et essaie de dire. 

**Q 6.2**: Utilité de la normalisation  

La normalisation permet de donner une liberté à l'utilisateur d'écrire son texte comme il veut, le programme s'occupe de filtrer et transformer le texte original en un texte lisible et compréhensible au chatbot. C'est plus facile de filtrer les mots-clés. 

Exemple: "la météo" ==> "la meteo", le mot "meteo" doit etre formatté correctement ou il ne sera pas détécté.

**Q 6.3**: Exemple d'utilisation de la mémoire  

Si l'utilisateur converse avec le chatbot durant plusieurs tours, le bot utilise la mémoire pour mieux répondre à l'utilisateur sans l'utilisateur ayant besoin de reformuler et de redonner des informations déja vus. Dans notre cas la mémoire couvre la météo, donc on peut demander la météo pour différentes instances de temps (matin, soir, dans quelques heures, passé, etc).

**Q 6.4**: Limites de l'approche par règles  

Il est impossible de déterminer des règles pour chaque situation et chaque problème (concernat le problème de conversation humaine), eventuellement il faut utiliser des modèles probabilistes pour essayer de simuler cette intelligence, et de créer un modèle capable de s'adapter aux situations et conversations. Certes le modèle est bien plus complexe, en utilisant des bases comme les transformeurs, Auto-encodeurs, combinaisons des ces méthodes et meme des agents intelligents pour mieux gérer le flux de conversation (plusieurs modèles spécialisés, avec un agent qui définit le contexte de la conversation et décide du meilleur LLM avec qui communiquer).  

Concernant des problèmes et solutions simples, on remarque que le contexte est limité à la taille du dictionnaire, donc il serait mieux de dynamiquement augmenter la taille de ce dictionnaire au fur et à mesure qu'on parle avec le chatbot, afin qu'il ait plus de précision. 

**Q 6.5**: Extension - Intention "blague"

In [11]:


INTENTS["blague"] = ["blague", "rigole", "rire", "drole", "humour", "marrant", "joke"]

BLAGUES = [
    "Blague 1: (c'est ca la blague)",
    "Blague 2: (je l'ai fait une fois, pourquoi pas deux?)",
    "Un poissonier dis à un farmeur: tu veux entendre un blague? Poisson d'avr- Blague 3 (c'est toujours ci drole)"
]
# new respond function
def respond(intent, message, memory):
    message = normalize(message)

    match intent:
        case "salutation":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return random.choice(INTENTS[intent])
        
        case "aurevoir":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return random.choice(INTENTS[intent])
        
        case "heure":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            now = datetime.now()
            return f"Il est {now.hour}:{now.minute:02d}"
        
        case "identite":
            memory["last_intent"] = intent
            memory["last_city"] = "casablanca"
            
            return "Bonjour, je suis votre Agent IA, capable de converser avec vous sans aucun soucis."
        
        case "blague":
            memory["last_intent"] = intent
            
            return random.choice(BLAGUES)
        
        case "meteo":
            ville = [villes[city] for city in message.split(sep=" ") if city in villes]
            if not ville:
                ville = "casablanca"
            else:
                ville = ville[0]
            memory["last_intent"] = intent
            memory["last_city"] = ville

            return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
        
        case "inconnu":
            if memory.get("last_intent") == "meteo":
                time_ids = INTENTS["heure"] 
            
                ville = [villes[city] for city in message.split(sep=" ") if city in villes]
                heure = [time_dict[heure] for heure in message.split(sep=" ") if heure in time_ids]

                if not ville:
                    ville = memory.get("last_city", "casablanca")
                else:
                    ville = ville[0]
                
                if heure:
                    return f"{ville} sera à {FAKE_WEATHER[ville]} {heure[0]}."
                else:
                    return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
            
            return "Désolé, je n'ai pas compris votre demande."

print("=== Tests de l'intention 'blague' ===\n")

test_memory = {}

test1 = "Raconte-moi une blague"
intent_test1 = detect_intent(test1)
response_test1 = respond(intent_test1, test1, test_memory)
print(f"Vous: {test1}")
print(f"Intent détecté: {intent_test1}")
print(f"Chatbot: {response_test1}")
print(f"Memory: {test_memory}\n")

test2 = "J'ai envie de rire"
intent_test2 = detect_intent(test2)
response_test2 = respond(intent_test2, test2, test_memory)
print(f"Vous: {test2}")
print(f"Intent détecté: {intent_test2}")
print(f"Chatbot: {response_test2}")
print(f"Memory: {test_memory}\n")

test3 = "Dis-moi quelque chose de drôle"
intent_test3 = detect_intent(test3)
response_test3 = respond(intent_test3, test3, test_memory)
print(f"Vous: {test3}")
print(f"Intent détecté: {intent_test3}")
print(f"Chatbot: {response_test3}")
print(f"Memory: {test_memory}\n")

=== Tests de l'intention 'blague' ===

Vous: Raconte-moi une blague
Intent détecté: blague
Chatbot: Un poissonier dis à un farmeur: tu veux entendre un blague? Poisson d'avr- Blague 3 (c'est toujours ci drole)
Memory: {'last_intent': 'blague'}

Vous: J'ai envie de rire
Intent détecté: blague
Chatbot: Blague 1: (c'est ca la blague)
Memory: {'last_intent': 'blague'}

Vous: Dis-moi quelque chose de drôle
Intent détecté: blague
Chatbot: Un poissonier dis à un farmeur: tu veux entendre un blague? Poisson d'avr- Blague 3 (c'est toujours ci drole)
Memory: {'last_intent': 'blague'}

